In [2]:
# %pip install transformers==4.28.1
# %pip install torch==2.1.2
# %pip install datasets==2.14.5

# 更多环境信息，可以查看 code/env.yml

In [3]:
# 导入必要包
import torch
from torch.utils.data import DataLoader, Dataset
import json
import datasets

In [4]:
# 下载数据
# https://github.com/gyfffffff/llm-deploy/releases/tag/IF_data
# 将下载的两个json文件放入同级目录data下

In [5]:
# 下载模型

# windows
# %pip install -U "huggingface-hub[cli]"
# !$env:HF_ENDPOINT = "https://hf-mirror.com"
# !huggingface-cli download --resume-download openai-community/gpt2 --local-dir ../../models/GPT-2

# linux
# %pip install -U "huggingface-hub[cli]"
# !export HF_ENDPOINT=https://hf-mirror.com
# !huggingface-cli download --resume-download openai-community/gpt2 --local-dir ../../models/GPT-2

In [7]:
# 准备数据
datapath = 'data/alpaca_gpt4_data_dev.json'  # dev数据只有少量数据，用于开发，实际训练时请使用full数据集
# datapath = 'data/alpaca_gpt4_data.json'  # full数据集

# 定义tokenizer
from transformers import GPT2Tokenizer
model_path = "../../models/GPT-2"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# 定义数据集类
def get_flat_data(datapath):
    data = json.load(open(datapath, 'r'))
    inputs = []  # 指令和输入
    labels = []  # 输出

    for item in data:
        inputs.append(f"{item['instruction']} {item['input']}")
        labels.append(item['output']) 
    return {
        'input': inputs,
        'label': labels
    }

def tokenize(example):
    tokenizer.pad_token = tokenizer.eos_token
    input_token = tokenizer(
        example['input'],
        max_length=512,
        truncation=True
    )
    iids = input_token['input_ids']
    label_token = tokenizer(
        example['label'],
        max_length=512,
        truncation=True
    )
    lids = label_token['input_ids']

    lengths = []
    input_ids = []
    label_ids = []
    attention_mask = []
    for iid, lid in zip(iids, lids):
        lengths.append(len(iid) + len(lid))
        input_ids.append(iid + lid)
        label_ids.append([-100]*len(iid) + lid)
        attention_mask.append([1]*(len(iid) + len(lid)))
    
    lengths = torch.tensor(lengths)
    pad_length = (lengths.max() - lengths).tolist()
    for i, l in enumerate(pad_length):
        input_ids[i] = [tokenizer.pad_token_id]*l + input_ids[i]
        attention_mask[i] = [0]*l + attention_mask[i]
        label_ids[i] = [-100]*l + label_ids[i]
    return {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        "labels": torch.tensor(label_ids, dtype=torch.long)
    }

# 创建数据集
flatdata = get_flat_data(datapath)
dataset = datasets.Dataset.from_dict(flatdata)
dataset = dataset.map(tokenize, batched=True, batch_size=len(dataset))  # 使用map方法对数据集进行批处理
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
    

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

In [8]:
# train

# 定义模型
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(model_path)

def train_step(batch):
    kwargs = {
        "input_ids": batch["input_ids"],
        "attention_mask": batch["attention_mask"],
        "labels": batch["labels"],
    }
    res = model(**kwargs)["loss"]
    return res

def val():
    model.eval()
    model.to('cpu')
    for step, batch in enumerate(dataloader):
        loss = train_step(batch)
        if step % 10 == 0:
            print(f"val step: {step}, loss: {loss.item()}")

def train():
    model.train()
    model.to('cpu')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    for epoch in range(3):
        for step, batch in enumerate(dataloader):
            loss = train_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if step % 10 == 0:
                print(f"epoch: {epoch}, step: {step}, loss: {loss.item()}")
        torch.save(model.state_dict(), f"model_{epoch}.pt")
        val()

train()

epoch: 0, step: 0, loss: 7.793501853942871


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:39                                                                                   │
│                                                                                                  │
│   36 │   │   torch.save(model.state_dict(), f"model_{epoch}.pt")                                 │
│   37 │   │   val()                                                                               │
│   38                                                                                             │
│ ❱ 39 train()                                                                                     │
│   40                                                                                             │
│                                                                                                  │
│ in train:30                                                                                      │
│                                                                                                  │
│   27 │   optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)                               │
│   28 │   for epoch in range(3):                                                                  │
│   29 │   │   for step, batch in enumerate(dataloader):                                           │
│ ❱ 30 │   │   │   loss = train_step(batch)                                                        │
│   31 │   │   │   loss.backward()                                                                 │
│   32 │   │   │   optimizer.step()                                                                │
│   33 │   │   │   optimizer.zero_grad()                                                           │
│                                                                                                  │
│ in train_step:13                                                                                 │
│                                                                                                  │
│   10 │   │   "attention_mask": batch["attention_mask"],                                          │
│   11 │   │   "labels": batch["labels"],                                                          │
│   12 │   }                                                                                       │
│ ❱ 13 │   res = model(**kwargs)["loss"]                                                           │
│   14 │   return res                                                                              │
│   15                                                                                             │
│   16 def val():                                                                                  │
│                                                                                                  │
│ e:\anaconda\envs\dl2\lib\site-packages\torch\nn\modules\module.py:1518 in _wrapped_call_impl     │
│                                                                                                  │
│   1515 │   │   if self._compiled_call_impl is not None:                                          │
│   1516 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1517 │   │   else:                                                                             │
│ ❱ 1518 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1519 │                                                                                         │
│   1520 │   def _call_impl(self, *args, **kwargs):                                                │
│   1521 │   │   forward_call = (self._slow_forward if torch._C._get_tracing_state() else self.fo  │
│                                                                                                  │
│ e:\anaconda\envs\dl2\lib\site-packages\torch\nn\modules\mod

In [32]:
# 测试效果
def test(trained=True):
    gpt2 = GPT2LMHeadModel.from_pretrained(model_path)
    if trained:
        gpt2.load_state_dict(torch.load('model_2.pt'))
    gpt2.eval()
    gpt2.to('cpu')
    
    text = "Please let me know your thoughts on the given place and why you think it deserves to be visited. \"Barcelona, Spain\""
    encoded_input = tokenizer(text, return_tensors='pt')
    output = gpt2(**encoded_input)
    logits = output.logits
    predicted_index = torch.argmax(logits, dim=-1)
    predicted_text = tokenizer.decode(predicted_index[0])
    print(predicted_text)

test(trained=True)
test(trained=False)

. me know if thoughts on this matter.. time you think it's to be here.

celona is Spain"

. me know if thoughts on this matter.. time you think it's to be here.

celona" Spain"

